<a href="https://colab.research.google.com/github/ufofon2/korea/blob/master/04_rag_embedding_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 환경변수 설정


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

### Pinecone 클라이언트 초기화


In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

### 서버리스 인덱스 생성


In [ ]:
index_name = "wiki"

# Pinecone에 있는 모든 인덱스를 순회합니다.
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [ ]:
# Pinecone 인덱스를 생성합니다.
# 인덱스 이름은 "wiki"이고, 차원은 1536, 메트릭은 코사인 유사도를 사용합니다.
# 인덱스는 AWS의 us-east-1 리전에서 서버리스 사양으로 생성됩니다.
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

In [ ]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

### 임베딩 객체 생성


In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 클래스를 초기화합니다.
# OPENAI_API_KEY는 환경 변수에서 가져온 API 키입니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)

### 텍스트 분할


In [ ]:
%pip install -qU datasets

In [ ]:
from datasets import load_dataset

# 'wikipedia' 데이터셋의 '20220301.simple' 버전을 로드합니다.
# 데이터셋의 'train' 스플릿에서 처음 100개의 샘플을 가져옵니다.
# trust_remote_code=True는 원격 코드 실행을 신뢰한다는 의미입니다.
data = load_dataset("wikipedia", "20220301.simple", split="train[:100]", trust_remote_code=True)

In [ ]:
data

In [ ]:
for record in data:
    print(record)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter를 초기화합니다.
# 이 클래스는 텍스트를 재귀적으로 문자 단위로 분할합니다.
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 분할할 텍스트의 크기
    chunk_overlap=20,  # 분할된 텍스트의 중첩 크기
    length_function=len,  # 텍스트 길이를 계산하는 함수
    separators=["\n\n", "\n", " ", ""]  # 분할할 텍스트의 구분자
)

### 레코드 업서트(Upsert)


In [ ]:
from tqdm.auto import tqdm
from uuid import uuid4
import time

batch_size = 100
texts = []
metas = []
count = 0

# 데이터셋의 각 샘플에 대해 반복합니다.
for i, sample in enumerate(tqdm(data)):

    full_text = sample["text"] # Wikipedia 문서 텍스트
    metadata = {
        'wiki_id': str(sample["id"]),  # Wikipedia 문서 ID
        'url': sample["url"],  # Wikipedia 문서 URL
        'title': sample["title"],  # Wikipedia 문서 제목
    }


    chunks = splitter.split_text(full_text)  # 텍스트를 청크로 분할합니다.
    # print(len(chunks))

    # 각 청크에 대해 반복합니다.
    for i, chunk in enumerate(chunks):
        record = {
            'chunk_id': i,  # 청크 ID
            'full_text': full_text,  # 전체 텍스트
            **metadata,  # 메타데이터 언패킹
        }

        texts.append(chunk)  # 청크를 텍스트 목록에 추가합니다.
        metas.append(record)  # 메타데이터를 메타데이터 목록에 추가합니다.

        count += 1  # 처리된 청크 수를 증가시킵니다.

        # batch_size만큼의 청크를 처리할 때마다 청크를 Pinecone 인덱스에 추가합니다.
        if count % batch_size == 0:
            # Pinecone 인덱스에 청크를 추가합니다.
            ids = [str(uuid4()) for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts)
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace="wiki-ns1")
            # 청크 목록과 메타데이터 목록을 비웁니다.
            texts = []
            metas = []
            # 1초 대기합니다.
            time.sleep(1)


